In [1]:
import sqlite3
import os
import numpy as np
from typing import List, Dict, Union, Optional
import faiss  # 确保已安装 faiss-cpu
from transformers import CLIPProcessor, CLIPModel # 确保已安装 transformers 和 pillow
from PIL import Image # 确保已安装 pillow
import torch # transformers 依赖 torch
import zhipuai # 导入 ZhipuAI 客户端
# import base64 # 用于可能的图像 base64 编码（备用，当前GLM-4-flash不需要）
# import io # 用于图像内存操作

import json # 导入 json 库用于读取数据文件
import time # 导入 time 库用于增加延迟，观察过程更清晰
import random # 导入 random 库用于随机选择图片进行查询


# --- 数据加载与图片关联函数 ---

def load_data_from_json_and_associate_images(json_path: str, image_dir: str) -> List[Dict]:
    """
    # 从 JSON 文件加载文档数据，并尝试在指定的图像目录中关联对应的图片文件。

    # JSON 文件中的每个对象有一个 'name' 字段，对应的图像文件名为 'name' + '.png' 或 '.jpg' 等。

    Args:
        json_path: 包含文档元数据的 JSON 文件路径。
        image_dir: 存放图片文件的目录路径。

    Returns:
        一个包含文档元数据的字典列表，每个字典包含 'id', 'text', 'image_path'。
    """
    # print(f"--- 正在加载数据并关联图像 ---")
    if not os.path.exists(json_path):
        # print(f"错误: 未找到 JSON 文件 '{json_path}'。")
        return []

    documents = []
    # 尝试几种常见的图像文件扩展名进行关联
    image_extensions = ['.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff']

    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
    except json.JSONDecodeError as e:
        # print(f"错误: JSON 文件 '{json_path}' 解析失败: {e}")
        return []
    except Exception as e:
        # print(f"错误: 读取 JSON 文件 '{json_path}' 失败: {e}")
        return []

    # print(f"已加载 {len(json_data)} 条记录来自 '{json_path}'。")

    for item in json_data:
        # 提取文档 ID 和文本内容
        # 使用 'name' 字段作为文档 ID
        doc_id = item.get('name')
        # 使用 'description' 字段作为文本内容
        text_content = item.get('description')

        # 跳过缺少关键字段的记录
        if not doc_id or not text_content:
            # print(f"警告: 跳过记录，缺少 'name' 或 'description': {item}") # 频繁输出警告，可以注释掉或改为计数
            continue

        # 尝试寻找关联的图像文件
        image_path = None
        # 确保图像目录存在且已指定
        if image_dir and os.path.exists(image_dir):
             # 遍历可能的图像扩展名，查找匹配的文件名
             for ext in image_extensions:
                 # 构建潜在的图像文件路径 (例如: images/Bandgap1.png)
                 potential_image_filename = str(doc_id) + ext
                 potential_image_path = os.path.join(image_dir, potential_image_filename)
                 if os.path.exists(potential_image_path):
                     image_path = potential_image_path
                     # print(f"  找到关联图像: {image_path}") # 可选：打印找到的图像路径
                     break
             # if not image_path:
             #      print(f"  未找到 ID '{doc_id}' 的关联图像。") # 可选：打印未找到图像的文档ID
        # else:
        #      print(f"警告: 图像目录 '{image_dir}' 不存在或未指定。") # 可选打印


        # 将提取的信息添加到文档列表中
        documents.append({
            'id': str(doc_id), # 确保ID是字符串类型
            'text': str(text_content) if text_content is not None else None, # 确保文本是字符串或None
            'image_path': image_path
        })

    # print(f"成功加载并准备了 {len(documents)} 个文档进行索引。")
    # print(f"--- 数据加载完毕 ---")
    return documents


# --- 1. 多模态编码器 (MultimodalEncoder) - 负责向量化 ---

class MultimodalEncoder:
    """
    # 使用 Hugging Face Transformers 的 CLIP 模型作为多模态编码器。
    # 负责将文本和/或图像转换为向量。
    """
    # __init__ 方法：初始化编码器，加载模型和处理器
    def __init__(self, model_name: str = "openai/clip-vit-base-patch32"):
        # CLIP模型还可以选择性地使用其他模型，如 "asus-uwk/distil-clip-vit-base-patch32"
        try:
            # 加载预训练的 CLIP 模型和处理器
            # CLIPProcessor 负责文本分词和图像预处理 (如缩放、归一化)
            self.processor = CLIPProcessor.from_pretrained(model_name)
            # CLIPModel 包含文本编码器和图像编码器
            self.model = CLIPModel.from_pretrained(model_name)
            # CLIP 模型的文本和图像编码器输出维度是相同的
            self.vector_dimension = self.model.text_model.config.hidden_size
            # print(f"MultimodalEncoder 初始化成功，使用模型: {model_name}, 向量维度: {self.vector_dimension}")
            # 切换到评估模式，不进行训练
            self.model.eval()
            # 检查是否有可用的 GPU，并将模型移至 GPU
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
                # print("CUDA 可用。将模型移至 GPU。")
            else:
                self.device = torch.device("cpu")
                # print("CUDA 不可用。使用 CPU。")
            self.model.to(self.device)

        except Exception as e:
             # print(f"加载 CLIP 模型 {model_name} 失败: {e}")
             # print("请确保已安装 'transformers', 'pillow', 和 'torch'。")
             raise e # 如果模型加载失败，阻止程序继续


    # encode 方法：对文本和/或图像进行编码
    def encode(self, text: Optional[str] = None, image_path: Optional[str] = None) -> np.ndarray:
        """
        # 使用 CLIP 模型对文本和/或图像进行编码。
        # 如果提供文本和图像，则分别编码并计算它们的平均向量作为联合表示。
        # 返回一个归一化的 numpy 向量。
        """
        if text is None and image_path is None:
            raise ValueError("必须提供文本或图像路径其中之一进行编码。")

        vectors = []

        # with torch.no_grad()：在推理模式下禁用梯度计算，减少内存和计算开销
        with torch.no_grad():
            if text is not None:
                # 编码文本
                # 将输入数据和模型一起移到同一个设备
                # processor(text=...) 处理文本输入
                # .to(self.device) 将数据移到 GPU/CPU
                text_inputs = self.processor(text=text, return_tensors="pt", padding=True, truncation=True).to(self.device)
                # self.model.get_text_features(...) 获取文本向量
                text_features = self.model.get_text_features(**text_inputs)
                # .squeeze().cpu().numpy() 转换维度，移回 CPU，转为 numpy array (float32)
                vectors.append(text_features.squeeze().cpu().numpy().astype('float32'))

            if image_path is not None:
                # 编码图像
                try:
                    # Image.open(...).convert("RGB") 使用 Pillow 打开并确保是 RGB 格式
                    image = Image.open(image_path).convert("RGB")
                    # 将输入数据和模型一起移到同一个设备
                    # processor(images=...) 处理图像输入
                    image_inputs = self.processor(images=image, return_tensors="pt").to(self.device)
                    # self.model.get_image_features(...) 获取图像向量
                    image_features = self.model.get_image_features(**image_inputs)
                    # .squeeze().cpu().numpy() 转换维度，移回 CPU，转为 numpy array (float32)
                    vectors.append(image_features.squeeze().cpu().numpy().astype('float32'))
                except FileNotFoundError:
                    # print(f"警告: 未找到图像文件 {image_path}。跳过图像编码。") # 编码时如果文件丢失，只打印警告
                    if text is None: # 如果图像是唯一的输入但文件不存在，则编码失败
                         raise FileNotFoundError(f"未找到图像文件 {image_path} 且未提供文本。")
                except Exception as e:
                    # print(f"处理图像 {image_path} 出错: {e}。跳过图像编码。") # 其他图像处理错误
                    if text is None: # 如果图像是唯一的输入且处理出错，则编码失败
                         raise RuntimeError(f"处理图像 {image_path} 出错且未提供文本。") from e


        if not vectors:
             # 如果是因为图像文件找不到或处理错误且没有文本输入导致 vectors 为空
             raise ValueError("编码失败: 未处理任何有效输入 (文本或图像)。")

        # 组合向量：如果同时有文本和图像，简单地取平均
        # 在实际应用中，更复杂的融合方法可能更有效
        combined_vector = np.mean(vectors, axis=0).astype('float32')

        # 归一化向量
        norm = np.linalg.norm(combined_vector)
        if norm > 1e-6: # 避免除以零
            combined_vector = combined_vector / norm
        else: # 如果向量接近零，可能意味着编码失败，返回零向量
             print("警告: 编码生成了一个接近零的向量。")
             combined_vector = np.zeros_like(combined_vector)


        return combined_vector


# --- 2. 索引器 (Indexer) - 负责文档处理、向量化、索引构建和数据存储 ---

class Indexer:
    """
    # 索引器：处理多模态文档，进行向量化，构建 Faiss 向量索引，并存储原始数据到 SQLite 数据库。
    # 负责管理文档数据和向量索引的存储，支持 Faiss 索引的持久化。
    """
    # __init__ 方法：初始化数据库和加载/创建 Faiss 索引
    def __init__(self, db_path: str = 'multimodal_rag_data.db', faiss_index_path: str = 'multimodal_rag_index.faiss', clip_model_name: str = "openai/clip-vit-base-patch32"):
        self.db_path = db_path
        self.faiss_index_path = faiss_index_path
        # Indexer 内部使用 Encoder 进行向量化
        # 创建 MultimodalEncoder 实例，Encoder 是 Indexer 的一部分
        self.encoder = MultimodalEncoder(clip_model_name)
        # 从编码器获取向量维度
        self.vector_dimension = self.encoder.vector_dimension

        # 先初始化数据库（存储元数据）
        self._init_db()
        # 再加载或创建 Faiss 索引（存储向量）
        self._load_faiss_index()


    # _init_db 方法：初始化 SQLite 数据库表
    def _init_db(self):
        """初始化 SQLite 数据库和表"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                # 创建 documents 表
                # vector_index_id 是 Faiss 索引内部使用的 ID，也作为数据库的主键
                # doc_id 是原始文档的 ID (例如 JSON 中的 name)，确保其唯一性
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS documents (
                        vector_index_id INTEGER PRIMARY KEY, -- 对应 Faiss IndexIDMap2 中的 ID
                        doc_id TEXT UNIQUE,                  -- 原始文档ID (例如 JSON 中的 'name')
                        text TEXT,                           -- 文本内容
                        image_path TEXT                      -- 图像文件路径
                    )
                ''')
                conn.commit()
                # print(f"数据库已初始化或连接成功，位置: {self.db_path}")
        except Exception as e:
             # print(f"初始化数据库失败: {e}")
             raise e # 数据库是核心依赖，失败则中断


    # _load_faiss_index 方法：加载或创建 Faiss 向量索引
    def _load_faiss_index(self):
        """加载或创建 Faiss 向量索引"""
        try:
            if os.path.exists(self.faiss_index_path):
                # 尝试加载现有索引
                # faiss.read_index 从文件加载索引对象
                self.index = faiss.read_index(self.faiss_index_path)
                # print(f"成功加载 Faiss 索引文件: {self.faiss_index_path}, 包含 {self.index.ntotal} 个向量。")
            else:
                # 文件不存在，创建一个新的 IndexIDMap2
                # print("未找到 Faiss 索引文件，将创建一个新的空索引。")
                self._create_new_faiss_index()

        except Exception as e:
            # print(f"加载 Faiss 索引文件失败: {e}")
            # print("将创建一个新的空索引作为回退。")
            # 加载失败，创建一个新的空索引作为回退
            self._create_new_faiss_index()

    # _create_new_faiss_index 方法：创建新的 Faiss 索引对象
    def _create_new_faiss_index(self):
         """创建一个新的 IndexIDMap2 索引"""
         # IndexFlatL2 作为基础索引（量化器）使用的是L2相似度； 这里换成 IndexFlatIP 也可以使用内积距离，也就是余弦相似度
         # quantizer = faiss.IndexFlatL2(self.vector_dimension) # L2距离
         quantizer = faiss.IndexFlatIP(self.vector_dimension) # 切换为内积距离 (余弦相似度，因为向量已归一化)
         # IndexIDMap2 允许我们使用自己的整数 ID (vector_index_id) 来添加和搜索向量
         self.index = faiss.IndexIDMap2(quantizer)
         # print(f"创建了新的空 Faiss IndexIDMap2 索引 (维度: {self.vector_dimension})。")
         print(f"创建了新的空 Faiss IndexIDMap2 索引 (维度: {self.vector_dimension}), 使用内积 (余弦相似度)。") # 更新打印信息


    def index_documents(self, documents: List[Dict]):
        """
        # 接收文档列表，进行编码、添加到向量索引和数据库。

        Args:
            documents: 一个字典列表，每个字典包含 'id', 'text', 'image_path'。
                       'text' 和 'image_path' 至少需要提供一个。
        """
        if not documents:
            # print("未提供文档进行索引。")
            return

        # print(f"正在处理 {len(documents)} 个文档进行索引...")
        vectors_to_add = [] # 收集成功编码的向量
        data_to_store = []  # 收集成功处理的文档元数据
        vector_ids_for_batch = [] # 收集与 vectors_to_add 对应的 vector_index_id

        # 在批次开始时确定本次添加的起始 vector_index_id
        # IndexIDMap2 的 ntotal 就是当前向量数量，可以作为下一个可用 ID 的起点
        # self.index.ntotal 是当前索引中已有的向量数量
        start_vector_index_id = self.index.ntotal
        # print(f"当前 Faiss 索引包含 {start_vector_index_id} 个向量。新文档将从 ID {start_vector_index_id} 开始分配。")

        conn = None # Initialize conn outside try block
        try:
            # 连接数据库，使用上下文管理器确保关闭
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            for i, doc in enumerate(documents):
                doc_id = doc.get('id')
                text = doc.get('text')
                image_path = doc.get('image_path')

                if not doc_id:
                    # print(f"跳过缺少 'id' 的文档: {doc}")
                    continue
                if not text and not image_path:
                     # print(f"跳过文档 {doc_id}，因为既没有文本也没有图像路径。")
                     continue

                # 检查 doc_id 是否已存在于数据库中以避免重复索引
                cursor.execute("SELECT vector_index_id FROM documents WHERE doc_id = ?", (doc_id,))
                if cursor.fetchone():
                     # print(f"文档 ID '{doc_id}' 已存在于数据库中。跳过。") # 太频繁，改为只计数
                     continue

                # 分配 vector_index_id for this document *before* encoding attempt
                # 如果编码失败，这个 ID 不会被实际添加到 Faiss 或 DB
                # 根据当前 Faiss 索引数量和当前批次已处理的文档数量，分配一个唯一的 vector_index_id
                current_vector_index_id = start_vector_index_id + len(vectors_to_add)

                try:
                    # 1. 多模态向量化 (使用 Encoder)
                    # 调用 Encoder 对文档进行编码
                    vector = self.encoder.encode(text=text, image_path=image_path)

                    # 如果编码成功
                    # 将向量、对应的 ID 和原始数据添加到批处理列表中
                    vectors_to_add.append(vector)
                    vector_ids_for_batch.append(current_vector_index_id)
                    data_to_store.append((current_vector_index_id, doc_id, text, image_path))

                except (FileNotFoundError, ValueError, RuntimeError, Exception) as e:
                    # 编码或处理失败，跳过此文档，不将其添加到 vectors_to_add 或 data_to_store
                    # print(f"编码或处理文档 {doc_id} (尝试分配 ID {current_vector_index_id}) 出错: {e}。跳过。")
                    continue # 继续处理下一个文档

            # 先收集所有成功编码的向量和数据，然后一次性添加到索引和数据库
            if not vectors_to_add:
                 # print("没有文档成功编码进行索引。")
                 return

            # 2. 向量索引构建 - 只添加成功编码的向量及其对应的 ID
            vectors_to_add_np = np.array(vectors_to_add, dtype='float32')
            # Faiss add_with_ids 方法需要 int64 类型的 ID 数组
            vector_ids_for_batch_np = np.array(vector_ids_for_batch, dtype='int64')

            # print(f"正在向 Faiss IndexIDMap2 添加 {len(vectors_to_add_np)} 个向量...")
            # index.add_with_ids method: Add vectors to the index, associating them with specified IDs
            self.index.add_with_ids(vectors_to_add_np, vector_ids_for_batch_np)
            # print(f"已向 Faiss 索引添加 {len(vectors_to_add_np)} 个向量。总向量数: {self.index.ntotal}")

            # 3. 原始数据存储 - 只存储成功编码的文档数据
            if data_to_store:
                # print(f"正在数据库中存储 {len(data_to_store)} 条文档记录...")
                # 使用 executemany 进行批量插入，效率更高
                # INSERT OR IGNORE 防止因 doc_id UNIQUE 约束冲突导致整个批次失败
                cursor.executemany(
                    "INSERT OR IGNORE INTO documents (vector_index_id, doc_id, text, image_path) VALUES (?, ?, ?, ?)",
                    data_to_store
                )
                conn.commit() # 提交数据库事务
                # print(f"数据库插入完成。成功插入 {cursor.rowcount} 条记录 (部分可能因 doc_id 重复被忽略)。")
            else:
                 # print("没有文档成功处理并存储到数据库。")
                 pass

        except Exception as e:
            # print(f"批次索引过程中发生错误: {e}")
            if conn:
                conn.rollback() # 如果批次内任何步骤失败则回滚数据库操作
        finally:
            if conn:
                conn.close() # 确保连接关闭

    # get_document_by_vector_index_id 方法：根据 Faiss 内部 ID 查询数据库获取元数据
    def get_document_by_vector_index_id(self, vector_index_id: int) -> Optional[Dict]:
        """根据向量索引ID从数据库获取原始文档数据"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                # 根据 vector_index_id 查询
                cursor.execute(
                    "SELECT doc_id, text, image_path FROM documents WHERE vector_index_id = ?",
                    (vector_index_id,)
                )
                row = cursor.fetchone()
                if row:
                    doc_id, text, image_path = row
                    # 返回原始数据和用于关联的 vector_index_id
                    return {'id': doc_id, 'text': text, 'image_path': image_path, 'vector_index_id': vector_index_id}
                return None
        except Exception as e:
             # print(f"从数据库根据 vector_index_id {vector_index_id} 获取文档出错: {e}")
             return None


    # get_index_count 方法：获取 Faiss 索引中的向量数量
    def get_index_count(self) -> int:
         """获取向量索引中的向量数量"""
         return self.index.ntotal if hasattr(self, 'index') else 0

    # get_document_count 方法：获取数据库中存储的文档数量
    def get_document_count(self) -> int:
         """获取数据库中存储的文档数量"""
         try:
             with sqlite3.connect(self.db_path) as conn:
                 cursor = conn.cursor()
                 cursor.execute("SELECT COUNT(*) FROM documents")
                 count = cursor.fetchone()[0]
                 return count if count is not None else 0
         except Exception as e:
              # print(f"从数据库获取文档数量出错: {e}")
              return 0 # 返回0表示无法获取数量

    # save_index 方法：将 Faiss 索引保存到文件，实现持久化
    def save_index(self):
        """保存 Faiss 索引到文件"""
        # 只有当索引存在且不为空时才保存
        if hasattr(self, 'index') and self.index.ntotal > 0:
            try:
                # faiss.write_index 将索引对象保存到文件
                faiss.write_index(self.index, self.faiss_index_path)
                # print(f"Faiss 索引已保存到: {self.faiss_index_path}")
            except Exception as e:
                # print(f"保存 Faiss 索引失败: {e}")
                pass # 修复 IndentationError：except 块后需要至少一个缩进的语句
        elif hasattr(self, 'index'):
             # print("Faiss 索引为空，跳过保存。")
             pass
        else:
             # print("Faiss 索引未初始化，跳过保存。")
             pass

    # close 方法：在程序结束时调用，确保索引被保存
    def close(self):
        """清理资源 - 主要用于保存索引"""
        # 调用 save_index 方法保存索引
        self.save_index()
        # SQLite 连接在方法内部管理，Faiss 对象本身通常不需要特殊关闭方法


# --- 3. 检索器 (Retriever) - 负责查询向量化、相似度搜索和上下文组织 ---

class Retriever:
    """
    # 检索器：处理查询，进行向量化，在 Indexer 的索引中搜索，提取原始内容，组织上下文。
    """
    # __init__ 方法：初始化检索器，获取 Indexer 的引用
    def __init__(self, indexer: Indexer):
        """
        初始化检索器。需要传入一个 Indexer 实例，以便访问其索引和数据库。
        """
        if indexer is None or not isinstance(indexer, Indexer):
             raise ValueError("必须向 Retriever 提供有效的 Indexer 实例。")

        # Retriever 持有 Indexer 的引用，通过 Indexer 访问 Encoder、Faiss 索引和数据库查找方法
        self.indexer = indexer
        self.encoder = self.indexer.encoder # 使用 Indexer 的 Encoder
        self.vector_dimension = self.indexer.vector_dimension
        self.index = self.indexer.index # 使用 Indexer 的 Faiss 索引

        if self.index is None or self.index.ntotal == 0:
             # print("警告: 提供的 Indexer 没有索引向量。检索将返回空结果。")
             pass # 警告但不阻止初始化
        else:
             # print(f"Retriever 初始化成功，使用 Indexer 的索引，包含 {self.index.ntotal} 个向量。")
             pass # 成功初始化


    # retrieve 方法：执行整个检索流程
    def retrieve(self, query: Union[str, Dict], k: int = 5) -> List[Dict]:
        """
        # 接收用户查询（文本或包含文本/图像路径的字典），执行检索。
        # 负责查询向量化、相似度搜索和上下文组织。

        Args:
            query: 用户查询，可以是文本字符串，或包含 'text' 和/或 'image_path' 的字典。
            k: 检索最相似的前 k 个文档。 # 这是控制召回数量 (Top-K) 的参数

        Returns:
            一个字典列表，包含检索到的文档数据（来自数据库），以及相似度/距离信息。
            [{'id': ..., 'text': ..., 'image_path': ..., 'vector_index_id': ..., 'score': ...}, ...]
            按相似度分数降序排序 (对于内积)。
        """
        if self.index is None or self.index.ntotal == 0:
            # print("错误: 向量索引未加载或为空。无法执行检索。")
            print("  - 错误: 向量索引未加载或为空。无法执行检索。")
            return []

        # print(f"正在检索与查询相关的文档 (k={k})...") # 打印移到 main 函数
        query_vector = None

        try:
            # 1. 查询向量化 (使用 Encoder)
            # 调用 Encoder 对查询进行编码，支持文本或多模态查询
            print("  - 正在编码查询...")
            if isinstance(query, str):
                query_vector = self.encoder.encode(text=query)
                print("  - 已编码文本查询。")
            elif isinstance(query, dict):
                 text = query.get('text')
                 image_path = query.get('image_path')
                 if not text and not image_path:
                      # print("错误: 查询字典必须包含 'text' 或 'image_path'。")
                      print("  - 错误: 查询字典必须包含 'text' 或 'image_path'。")
                      return []
                 query_vector = self.encoder.encode(text=text, image_path=image_path)
                 print(f"  - 已编码多模态查询 (包含文本={text is not None}, 包含图像='{os.path.basename(image_path)}' if image_path else '无')。") # 打印图像文件名
            else:
                # print(f"不支持的查询类型: {type(query)}。查询必须是 str 或 dict。")
                print(f"  - 不支持的查询类型: {type(query)}。查询必须是 str 或 dict。")
                return []
            print("  - 查询编码完成。")

        except (FileNotFoundError, ValueError, RuntimeError, Exception) as e:
            # print(f"编码查询出错: {e}")
            print(f"  - 编码查询出错: {e}")
            return []

        # Faiss 搜索需要形状为 (n_queries, dim)，这里只有1个查询
        query_vector = query_vector.reshape(1, self.vector_dimension)

        # 2. 相似度检索 (使用 Indexer 的 Faiss IndexIDMap2 索引)
        # index.search(query_vector, k) 执行相似度搜索
        # D 是得分矩阵，I 是检索到的向量对应的 ID 矩阵
        # 这就是实现 Top-K 召回的地方！
        # 如果使用 IndexFlatIP (内积), 返回的是内积得分，越大越相似
        # 如果使用 IndexFlatL2 (L2距离), 返回的是距离，越小越相似 (Faiss会自动返回距离最小的)
        print(f"  - 正在 Faiss 索引中搜索 Top {k} 个相似向量...")
        scores, vector_index_ids = self.index.search(query_vector, k) # 将 distances 改为 scores
        print(f"  - Faiss 搜索找到 {len(vector_index_ids[0])} 个潜在最近邻。")

        retrieved_docs = []
        # 3. 根据检索到的向量索引ID，从数据库提取原始文档数据 (使用 Indexer 的方法) 并组织上下文
        # vector_index_ids 是一个二维数组，[0] 获取第一个（也是唯一一个）查询结果的 ID 列表
        print("  - 正在根据检索到的向量ID获取文档元数据...")
        for i, vector_index_id in enumerate(vector_index_ids[0]):
            # Faiss 在结果不足 k 个时可能返回 -1 作为填充
            if vector_index_id == -1:
                continue

            # 使用 Indexer 的方法根据 Faiss 内部存储的 ID (即 vector_index_id) 查找文档
            # 需要确保传递给 get_document_by_vector_index_id 的 ID 是整数类型
            doc_data = self.indexer.get_document_by_vector_index_id(int(vector_index_id))
            if doc_data:
                # 添加得分信息
                # 如果使用 IndexFlatIP (内积), scores 里存的是相似度分数，分数越大越相似
                # 如果使用 IndexFlatL2 (L2距离), scores 里存的是距离，距离越小越相似
                doc_data['score'] = float(scores[0][i]) # 添加 'score' 键，存储相似度得分
                retrieved_docs.append(doc_data)
                # print(f"  检索到文档 ID: {doc_data.get('id', 'N/A')} (向量索引 ID: {vector_index_id}, 得分: {doc_data['score']:.4f})") # 更新打印信息，已移到主函数中的打印函数
            else:
                print(f"  - 警告: 未在数据库中找到向量索引 ID 为 {vector_index_id} 的文档数据。可能索引或数据库不同步。")

        # 4. 结果排序 (Faiss Flat/IDMap 索引通常已按得分/距离返回排序结果)
        # retrieved_docs 已经按得分排序，因为 Faiss 返回的结果就是排序的
        # 对于 IndexFlatL2 是按距离升序；对于 IndexFlatIP 是按内积降序。
        # Generator 需要的是最相关的文档，Faiss 返回的顺序就是最相关的在前。
        # 我们返回文档字典列表。这个列表是 Generator 的上下文。
        print("  - 文档元数据获取完成。")
        return retrieved_docs


    def close(self):
        """清理资源 - Retriever 依赖于 Indexer，通常无需独立清理"""
        pass


# --- 4. 生成器 (Generator) - 负责提示工程、响应生成和后处理 ---

class Generator:
    """
    # 使用 ZhipuAI 客户端调用大语言模型。
    # 负责提示工程、响应生成和后处理。
    # API Key 从环境变量 ZHIPUAI_API_KEY 中加载。
    """
    # __init__ 方法：初始化智谱 AI 客户端
    def __init__(self, api_key: Optional[str] = None, model_name: str = "glm-4-flash"):
        # 初始化 ZhipuAI 客户端
        # 优先使用传入的 api_key，否则从环境变量 ZHIPUAI_API_KEY 获取
        # 获取智谱 AI API Key
        final_api_key = api_key if api_key else os.getenv("ZHIPUAI_API_KEY")

        if not final_api_key:
            raise ValueError("未提供 ZHIPUAI_API_KEY，且未在环境变量中找到。请设置 ZHIPUAI_API_KEY 环境变量或在 Generator 初始化时传入。")

        try:
            # 初始化 ZhipuAI 客户端
            self.client = zhipuai.ZhipuAI(api_key=final_api_key)
            self.model_name = model_name
            # print(f"Generator 初始化成功，使用智谱 AI 模型: {self.model_name}") # 打印移到 main 函数
        except Exception as e:
             # print(f"初始化智谱 AI 客户端失败: {e}")
             # 根据 zhipuai 库的异常类型，可以捕获特定错误
             raise e # API 客户端初始化失败是致命错误

    # generate 方法：构建 Prompt，调用 LLM，返回响应
    def generate(self, query: str, context: List[Dict]) -> str:
        """
        # 接收用户查询和检索到的上下文，构建 Prompt，调用 LLM 生成答案。

        Args:
            query: 原始用户查询字符串。
            context: 从 Retriever 获取的文档上下文列表。
                     每个元素是字典，如 {'id': ..., 'text': ..., 'image_path': ..., 'score': ...}

        Returns:
            LLM 生成的文本响应。
        """
        # print(f"正在使用 {len(context)} 个上下文文档生成对查询 '{query}' 的响应。") # 打印移到 main 函数

        # 1. 构建 Prompt (提示工程)
        # GLM-4-flash 是文本模型，上下文构建方式与原代码相同
        # 调用 _build_messages 构建 LLM API 需要的消息列表
        print("  - 正在构建发送给 LLM 的 Prompt...")
        messages = self._build_messages(query, context)
        print("  - Prompt 构建完成。")
        # print("  - Prompt 预览 (仅前两条消息):", messages[:2]) # 可选：打印Prompt预览

        # 2. 调用大语言模型 API (响应生成)
        print(f"  - 正在调用智谱 AI API (模型: {self.model_name})...")
        try:
            # 调用 ZhipuAI 客户端的 chat.completions.create 方法
            response = self.client.chat.completions.create(
                model=self.model_name, # 指定使用的模型
                messages=messages,     # 传入构建好的消息列表 (Prompt)
                temperature=0.7,       # 控制生成随机性 (0.0 - 1.0 或更高，取决于模型)
                max_tokens=1024        # 控制生成响应的最大长度
            )
            llm_response = response.choices[0].message.content
            print("  - 智谱 AI API 调用成功。")

        except zhipuai.APIStatusError as e:
             # print(f"智谱 AI API 状态错误: {e.response}")
             print(f"  - 错误: 智谱 AI API 状态错误: {e.response.status_code} - {e.response.text}") # 更详细的错误信息
             return f"调用 LLM 出错: {e.response.text}" # 返回错误信息给用户
        except zhipuai.APIConnectionError as e:
             # print(f"智谱 AI API 连接错误: {e}")
             print(f"  - 错误: 智谱 AI API 连接错误: {e}")
             return f"调用 LLM 出错: {e}" # 返回错误信息给用户
        except Exception as e:
             # print(f"调用 LLM 时发生意外错误: {e}")
             print(f"  - 错误: 调用 LLM 时发生意外错误: {e}")
             return f"调用 LLM 出错: {e}" # 返回错误信息给用户


        # 3. 响应后处理
        # 调用 _postprocess_response 对原始 LLM 输出进行后处理
        print("  - 正在进行响应后处理...")
        processed_response = self._postprocess_response(llm_response)
        print("  - 响应后处理完成。")

        return processed_response
    
    
    # _build_messages 方法：根据查询和上下文构建发送给 LLM 的消息
    def _build_messages(self, query: str, context: List[Dict]) -> List[Dict]:
        """
        # 构建用于 ZhipuAI Chat API 的消息列表 (针对文本模型 GLM-4-flash 系列)。
        # 旨在提供清晰的指令和格式化的上下文，帮助 LLM 理解和回答用户查询。
        """
        # System 消息：设定 LLM 的角色、规则，并提供格式化的上下文
        # System 消息包含对 LLM 的指令和检索到的文档上下文
        system_message_content = """
        你是一个专业的文档分析和问答助手。请严格根据以下提供的"参考文档"内容来回答用户的查询。

        # 重要规则:
        - **严格性:** 你的回答必须完全基于下面 "参考文档" 部分提供的信息。不要利用你已有的知识或其他来源。
        - **信息不足:** 如果 "参考文档" 中的信息不足以回答用户的查询，请明确说明"根据提供的文档，我无法回答这个问题"。
        - **图片理解限制:** 你无法直接查看图片。你对图片的理解**完全依赖于**文档中提供的文本描述（例如，文档的文本内容或关联的图片文件名）。
        - **图片查询处理:** 如果用户要求找出或描述具有某种特点的图片，请仔细阅读 "参考文档" 中每个文档的**文本内容**，并结合关联的图片文件名，来判断哪个文档（或哪些文档）描述了符合用户要求的图片。
        - **图片回答格式:** 如果你通过阅读文本确定某个文档关联的图片符合用户要求，请在你的回答中清晰地列出该图片的**文档ID**或**文件名**。

        # 参考文档:
        --- 开始参考文档 ---
        """.strip() # 使用 strip() 移除开头和结尾的空白行

        context_text_parts = [] # 用于收集每个检索到文档的格式化文本信息

        if not context:
            # 如果没有检索到任何文档，在上下文部分说明
            context_text_parts.append("未找到相关文档。")
        else:
            # 遍历检索到的文档列表，格式化每个文档的信息
            # 每个文档字典包含 'id', 'text', 'image_path', 'score'
            for i, doc in enumerate(context):
                # 格式化当前文档的信息，包括序号、ID、得分、文本和图片（如果存在）
                # Score (得分) 在这里是余弦相似度，越高越相似
                doc_id = doc.get('id', 'N/A')
                score = doc.get('score', 'N/A')
                text_content = doc.get('text', '无文本内容')
                image_path = doc.get('image_path')
                image_info = f"图像: [关联图片文件: {os.path.basename(image_path)}]" if image_path else "图像: [无关联图片]" # 显示图片文件名或无关联图片提示

                # 将文档信息添加到 parts 列表中
                context_text_parts.append(f"文档 {i+1} (ID: {doc_id}, 得分: {score:.4f}):") # 包含文档ID和得分
                # 将文档文本截断，避免上下文过长，同时保留关键信息
                truncated_text = text_content[:500] + ('...' if len(text_content) > 500 else '')
                context_text_parts.append(f"文本: {truncated_text}")
                context_text_parts.append(image_info) # 添加图片信息
                context_text_parts.append("-" * 10) # 分隔不同文档

            # 移除最后一个分隔线，并添加结束标记
            if context_text_parts and context_text_parts[-1] == "-" * 10:
                context_text_parts.pop()
            context_text_parts.append("--- 结束参考文档 ---")


        # 将 System 消息和格式化的上下文合并到 System 消息中
        # 将用户的原始查询作为 User 消息
        # 构建最终的消息列表格式，符合智谱 AI Chat API 要求
        messages = [
            {"role": "system", "content": system_message_content + "\n" + "\n".join(context_text_parts)}, # System 消息包含指令和格式化的上下文
            {"role": "user", "content": query} # User 消息是原始查询
        ]

        # 注意：如果使用的是 GLM-4V 等多模态模型，消息结构会不同，
        # user 角色 content 会是包含 text 和 image_url 对象的列表，
        # 且 image_url 需要提供图像的 base64 编码或公共 URL。
        # 当前示例使用 GLM-4-flash-250414 (文本模型)，因此保留文本上下文结构。

        return messages


    def _postprocess_response(self, llm_response: str) -> str:
        """对LLM的原始输出进行后处理"""
        # 当前只进行了简单的去首尾空白字符处理
        processed_response = llm_response.strip()
        # 在实际应用中，可以进行更复杂的后处理，例如：
        # 移除 LLM 可能产生的固定前缀/后缀
        # 将特定格式（如 Markdown）转换为 HTML 或纯文本
        # 检查并修复输出格式错误等
        return processed_response


# --- 示例使用流程 ---

if __name__ == "__main__":
    # --- 定义配置参数 ---
    json_data_path = 'data.json' # 原始 JSON 数据文件路径
    image_directory_path = 'images' # 图像文件存放目录路径
    db_file = 'multimodal_rag_data.db' # SQLite 数据库文件路径，用于存储文档元数据
    faiss_index_file = 'multimodal_rag_index.faiss' # Faiss 索引文件路径，用于存储向量并实现持久化

    CLIP_MODEL = "openai/clip-vit-base-patch32" # 多模态编码器 (CLIP) 模型名称 (可替换为其他 CLIP 模型)
    LLM_MODEL = "glm-4-flash-250414" # 智谱 AI 大语言模型名称，使用指定的模型版本

    # --- 清理旧文件 (为了演示方便，每次运行都重新创建数据库和索引) ---
    print("\n" + "="*50 + "\n")
    print("--- 清理旧的数据库和索引文件 ---")
    if os.path.exists(db_file):
        try:
            os.remove(db_file)
            print(f"已清理旧数据库文件: {db_file}")
        except OSError as e:
             print(f"清理旧数据库文件 {db_file} 失败: {e}")

    if os.path.exists(faiss_index_file):
        try:
            os.remove(faiss_index_file)
            print(f"已清理旧 Faiss 索引文件: {faiss_index_file}")
        except OSError as e:
             print(f"清理旧 Faiss 索引文件 {faiss_index_file} 失败: {e}")
    print("--- 清理完成 ---")
    print("\n" + "="*50 + "\n")

    # --- 1. 加载数据并关联图片 ---
    print("--- 步骤 1: 加载数据并关联图片 ---")
    # 调用 load_data_from_json_and_associate_images 函数加载原始数据
    #  它会读取 data.json 并尝试在 images 目录中找到对应的图片
    documents_to_index = load_data_from_json_and_associate_images(json_data_path, image_directory_path)

    # 检查是否成功加载到文档
    if not documents_to_index:
        print("\n" + "="*50 + "\n")
        print("错误：未加载到任何需要索引的文档。程序退出。")
        print("请检查 data.json 文件是否存在且格式正确，以及 images 目录是否存在且包含与 name 匹配的图片。")
        print("="*50 + "\n")
        exit() # 如果没有文档，后续步骤无法进行，直接退出

    print("--- 步骤 1 完成 ---")
    print("\n" + "="*50 + "\n")


    # --- 2. 初始化并运行 Indexer (构建索引和存储元数据) ---
    print("--- 步骤 2: 初始化并运行 Indexer ---")
    indexer = None # 初始化 indexer 为 None
    try:
        # 初始化 Indexer 实例
        #  它会初始化数据库，并加载或创建一个新的 Faiss 索引 (支持持久化)
        #  Indexer 内部会创建使用内积 (余弦相似度) 的 Faiss 索引
        print("  - 正在初始化 Indexer...")
        indexer = Indexer(db_path=db_file, faiss_index_path=faiss_index_file, clip_model_name=CLIP_MODEL)
        print("  - Indexer 初始化完成。")

        # 使用加载的数据进行索引
        # 调用 indexer.index_documents 方法，对加载的文档进行编码并添加到索引和数据库
        print("  - 正在对加载的文档进行索引...")
        indexer.index_documents(documents_to_index)
        print("  - 文档索引过程完成。")


        # 检查索引和数据库中的文档数量
        indexed_count = indexer.get_index_count()
        db_count = indexer.get_document_count()
        print(f"\nIndexer 总结：索引中的总向量数: {indexed_count}，数据库中的总文档数: {db_count}")

        # 如果索引后向量数量为零，则认为 Indexer 初始化失败或索引过程失败
        if indexed_count == 0:
             print("错误：索引中没有添加任何向量。Retriever 将无法工作。")
             indexer = None # 使 indexer 变量失效，阻止后续依赖它的步骤进行

    except Exception as e:
         # 捕获 Indexer 初始化或运行过程中的异常
         print(f"初始化或运行 Indexer 失败: {e}")
         indexer = None # 使 indexer 变量失效

    print("--- 步骤 2 完成 ---")
    print("\n" + "="*50 + "\n")


    # --- 3. 初始化并运行 Retriever (执行搜索) ---
    print("--- 步骤 3: 初始化并运行 Retriever ---")
    retriever = None # 初始化 retriever 为 None
    # 只有当 Indexer 成功初始化并且 Faiss 索引中有向量时，才能初始化 Retriever
    if indexer is not None and indexer.get_index_count() > 0:
        try:
            # 初始化 Retriever 实例，将 Indexer 实例传递进去
            #  Retriever 需要 Indexer 来访问 Faiss 索引和数据库查找方法
            print("  - 正在初始化 Retriever...")
            retriever = Retriever(indexer=indexer)
            print("  - Retriever 初始化成功。")
        except Exception as e:
             # 捕获 Retriever 初始化异常
             print(f"初始化 Retriever 失败: {e}")
             retriever = None # 使 retriever 变量失效
    else:
         # Indexer 不可用或索引为空时，跳过 Retriever 初始化
         print("Indexer 未成功初始化或没有索引向量。跳过 Retriever 初始化。")

    print("--- 3 步骤完成 ---")
    print("\n" + "="*50 + "\n")


    # --- 4. 初始化并运行 Generator (生成响应) ---
    print("--- 步骤 4: 初始化并运行 Generator ---")
    generator = None # 初始化 generator 为 None
    # 检查智谱 AI API Key 环境变量是否已设置
    zhipuai_api_key_from_env = os.getenv("ZHIPUAI_API_KEY")

    if not zhipuai_api_key_from_env:
        # 如果 API Key 未设置，打印错误并跳过 Generator 初始化和生成步骤
        print("错误: ZHIPUAI_API_KEY 环境变量未设置。Generator 未初始化。")
        print("请设置环境变量后重试（例如在终端执行 export ZHIPUAI_API_KEY='your_api_key'）")
    else:
        try:
            # 初始化 Generator 实例，使用智谱 AI API Key 和指定的模型
            print("  - 正在初始化 Generator...")
            generator = Generator(api_key=zhipuai_api_key_from_env, model_name=LLM_MODEL) # 使用指定的 LLM_MODEL
            print(f"  - Generator 初始化成功，使用模型: {LLM_MODEL}")
        except Exception as e:
             # 捕获 Generator 初始化异常
             print(f"初始化 Generator 失败: {e}")
             generator = None # 使 generator 变量失效

    print("--- 步骤 4 完成 ---")
    print("\n" + "="*50 + "\n")

    # --- 5. 执行查询并生成响应 (仅当 Retriever 和 Generator 都可用时) ---
    print("--- 步骤 5: 执行查询并生成响应 ---")

    if retriever and generator: # 确保 Retriever 和 Generator 都已成功初始化

        # --- Helper function to print retrieved documents clearly ---
        # 定义一个辅助函数，用于清晰地打印检索到的文档信息，包括得分 (余弦相似度)
        def print_retrieved_docs(docs: List[Dict]):
            if not docs:
                print("    (未检索到文档)")
                return
            # 遍历检索到的文档列表
            for i, doc in enumerate(docs):
                # 打印文档序号、ID、得分、文本片段和图像文件（如果存在）
                #  得分即为余弦相似度 (因为向量已归一化且使用IndexFlatIP)
                score_str = f"得分: {doc.get('score', 'N/A'):.4f}" if 'score' in doc else "无得分信息"
                print(f"    {i+1}. ID: {doc.get('id', 'N/A')}, {score_str}")
                # 打印文本内容的前150个字符加...
                text_preview = doc.get('text', '无文本')
                print(f"       文本: {text_preview[:150]}{'...' if len(text_preview) > 150 else ''}")
                if doc.get('image_path'):
                    print(f"       图像文件: {os.path.basename(doc['image_path'])}") # 只显示文件名
                print("    " + "-" * 20) # 分隔线


        # --- 定义更多查询示例 ---
        # 你可以根据你的 data.json 和 images 目录内容自定义这些查询，使其更贴合实际数据
        text_queries = [
            "带隙基准电路的核心原理是什么？", # 纯文本查询 1
            "什么是PTAT电流以及它在带隙电路中的作用？", # 纯文本查询 2
            "CTAT电压是如何补偿温度变化的？", # 纯文本查询 3
            "哪些文档讨论了使用MOS管构建的带隙电路？", # 纯文本查询 4 (可能根据描述找到相关文档)
            "能否总结一下不同带隙电路实现方案的差异？" # 纯文本查询 5
        ]

        # 尝试获取所有带有实际关联图片的文件路径及其对应的文档 ID
        image_docs_available = [(doc['id'], doc['image_path']) for doc in documents_to_index if doc.get('image_path') and os.path.exists(doc['image_path'])]

        if not image_docs_available:
            print("\n警告: 未找到任何带有实际关联图像的文档。将跳过图片和多模态查询示例。")
            # 清空查询列表以跳过循环
            image_queries_data = []
            multimodal_queries_data = []
        else:
            # 随机选择 5 个带图片的文档用于生成图片和多模态查询示例
            num_queries_to_generate = min(5, len(image_docs_available))
            selected_image_docs = random.sample(image_docs_available, num_queries_to_generate)

            image_queries_data = []
            multimodal_queries_data = []

            for doc_id, img_path in selected_image_docs:
                img_filename = os.path.basename(img_path)

                # 纯图像查询示例数据
                image_queries_data.append({
                    'image_path': img_path,
                    'query_text_for_generator': f"请根据相关文档，描述与文件 '{img_filename}' 关联的图片及其主要信息。", # 纯图像查询，附带给 Generator 的文本提示
                    'description': f"关于图片文件 '{img_filename}' 的信息" # 用于打印的查询描述
                })

                # 多模态查询示例数据 (结合图片和文本问题)
                multimodal_queries_data.append({
                    'text': f'请结合这张图片（文件: {img_filename}）和相关的文档内容，详细描述这个电路的主要功能和特点。', # 查询文本部分
                    'image_path': img_path, # 查询图像路径部分
                     'description': f"结合图片 '{img_filename}' 和相关文本，描述电路功能特点" # 用于打印的查询描述
                })


        # --- 逐个执行纯文本查询 ---
        print("\n" + "#"*50 + "\n")
        print(f">>> 开始执行纯文本查询示例 ({len(text_queries)} 次) <<<")
        print("#"*50 + "\n")
        for i, query in enumerate(text_queries):
            print(f"\n--- 文本查询 {i+1}/{len(text_queries)} ---")
            print(f"用户查询: {query}")

            try:
                # 调用 retriever.retrieve 执行检索 (召回 5 个文档)
                print("  - 正在执行检索...")
                retrieved_context = retriever.retrieve(query, k=5)
                print(f"  - 检索到 {len(retrieved_context)} 个文档。详细信息如下:")
                # 打印检索到的文档信息（包括得分/余弦相似度）
                print_retrieved_docs(retrieved_context)

                # 调用 Generator 生成响应
                if retrieved_context:
                    print("\n  - 正在使用检索结果生成响应...")
                    generated_response = generator.generate(query, retrieved_context)
                    print("\n  生成的响应:")
                    print(generated_response)
                else:
                     print("\n  未检索到相关文档，跳过响应生成。")

            except Exception as e:
                 print(f"\n  执行查询或生成响应时出错: {e}")

            print(f"\n--- 文本查询 {i+1}/{len(text_queries)} 结束 ---")
            if i < len(text_queries) - 1:
                print("\n" + "-"*50 + "\n")
                time.sleep(1) # 增加短暂延迟，观察过程更清晰

        print("\n" + "#"*50 + "\n")
        print(">>> 纯文本查询示例结束 <<<")
        print("#"*50 + "\n")
        time.sleep(2) # 增加延迟，分隔不同类型的查询


        # --- 逐个执行纯图像查询 ---
        print("\n" + "#"*50 + "\n")
        print(f">>> 开始执行纯图像查询示例 ({len(image_queries_data)} 次) <<<")
        print("#"*50 + "\n")
        if image_queries_data:
            for i, query_data in enumerate(image_queries_data):
                print(f"\n--- 纯图像查询 {i+1}/{len(image_queries_data)} ---")
                image_path = query_data['image_path']
                query_text_for_generator = query_data['query_text_for_generator']
                query_description = query_data['description']

                print(f"用户查询 (纯图像): {query_description}")
                print(f"传递给Generator的文本提示: {query_text_for_generator}")

                try:
                    # 调用 retriever.retrieve 执行检索 (召回 5 个文档)
                    print("  - 正在执行检索...")
                    retrieved_context = retriever.retrieve({'image_path': image_path}, k=5) # Retriever 只接收包含 image_path 的字典
                    print(f"  - 检索到 {len(retrieved_context)} 个文档。详细信息如下:")
                    # 打印检索到的文档信息（包括得分/余弦相似度）
                    print_retrieved_docs(retrieved_context)

                    # 调用 Generator 生成响应
                    if retrieved_context:
                        print("\n  - 正在使用检索结果生成响应...")
                        # Generator (GLM-4-flash) 只能根据检索到的文档文本来回答关于图片的问题。
                        # 它会查找上下文中文本是否提及了与这张图片相似或相关的描述。
                        generated_response = generator.generate(query_text_for_generator, retrieved_context) # 使用预设的文本提示和检索到的文本上下文
                        print("\n  生成的响应:")
                        print(generated_response)
                    else:
                         print("\n  未检索到相关文档，跳过响应生成。")

                except Exception as e:
                     print(f"\n  执行查询或生成响应时出错: {e}")

                print(f"\n--- 纯图像查询 {i+1}/{len(image_queries_data)} 结束 ---")
                if i < len(image_queries_data) - 1:
                    print("\n" + "-"*50 + "\n")
                    time.sleep(1) # 增加短暂延迟

        else:
            print("\n  (由于没有找到足够的带关联图像的文档，跳过此示例)")

        print("\n" + "#"*50 + "\n")
        print(">>> 纯图像查询示例结束 <<<")
        print("#"*50 + "\n")
        time.sleep(2) # 增加延迟


        # --- 逐个执行多模态查询 ---
        print("\n" + "#"*50 + "\n")
        print(f">>> 开始执行多模态查询示例 ({len(multimodal_queries_data)} 次) <<<")
        print("#"*50 + "\n")
        if multimodal_queries_data:
             for i, query_data in enumerate(multimodal_queries_data):
                print(f"\n--- 多模态查询 {i+1}/{len(multimodal_queries_data)} ---")
                text_part = query_data['text']
                image_part = query_data['image_path']
                query_description = query_data['description']

                print(f"用户查询 (多模态): {query_description}")
                print(f"查询文本部分: '{text_part}'")
                print(f"查询图像文件: '{os.path.basename(image_part)}'")


                try:
                    # 调用 retriever.retrieve 执行检索 (召回 5 个文档)
                    # CLIP 编码器会将文本和图片融合编码为向量，然后在多模态向量空间中寻找最相似的文档。
                    print("  - 正在执行检索...")
                    retrieved_context = retriever.retrieve(query_data, k=5) # Retriever 接收多模态查询字典
                    print(f"  - 检索到 {len(retrieved_context)} 个文档。详细信息如下:")
                    # 打印检索到的文档信息（包括得分/余弦相似度）
                    print_retrieved_docs(retrieved_context)

                    # 调用 Generator 生成响应
                    if retrieved_context:
                        print("\n  - 正在使用检索结果生成响应...")
                        # Generator (GLM-4-flash) 只能根据检索到的文档文本来回答。
                        # 它会结合查询的文本部分和检索到的文档文本来生成答案。
                        generated_response = generator.generate(text_part, retrieved_context) # Generator 接收查询的文本部分和检索到的文本上下文
                        print("\n  生成的响应:")
                        print(generated_response)
                    else:
                         print("\n  未检索到相关文档，跳过响应生成。")

                except Exception as e:
                     print(f"\n  执行查询或生成响应时出错: {e}")


                print(f"\n--- 多模态查询 {i+1}/{len(multimodal_queries_data)} 结束 ---")
                if i < len(multimodal_queries_data) - 1:
                    print("\n" + "-"*50 + "\n")
                    time.sleep(1) # 增加短暂延迟

        else:
             print("\n  (由于没有找到足够的带关联图像的文档，跳过此示例)")


        print("\n" + "#"*50 + "\n")
        print(">>> 多模态查询示例结束 <<<")
        print("#"*50 + "\n")


    else:
        # 如果 Retriever 或 Generator 未成功初始化，则跳过所有查询演示
        print("\n跳过查询和生成步骤：")
        if not retriever:
            print("- Retriever 未成功初始化 (请检查 Indexer 步骤是否成功)。")
        if not generator: # 这里检查 generator 是否为 None
             print(f"- Generator 未成功初始化 (请检查 ZHIPUAI_API_KEY 环境变量或模型 '{LLM_MODEL}' 是否可用)。")

    print("--- 步骤 5 完成 ---")
    print("\n" + "="*50 + "\n")


    # --- 6. 清理资源 ---
    print("--- 步骤 6: 清理资源 ---")
    # 调用 indexer.close() 方法。它会负责将当前内存中的 Faiss 索引保存到文件。
    if indexer:
        indexer.close()
        print("Indexer 资源已清理 (Faiss 索引已保存)。")
    # Retriever 和 Generator 在当前设置下通常无需特殊关闭方法
    print("Retriever 和 Generator 资源已清理。")
    print("--- 清理完成 ---")
    print("\n" + "="*50 + "\n")


    # --- 示例执行完毕 ---
    print("\n示例执行完毕。")
    print(f"您可以在当前目录下找到数据库文件 '{db_file}' 和 Faiss 索引文件 '{faiss_index_file}'。")

d:\ProgramData\anaconda3\envs\mrag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.




--- 清理旧的数据库和索引文件 ---
已清理旧数据库文件: multimodal_rag_data.db
--- 清理完成 ---


--- 步骤 1: 加载数据并关联图片 ---
--- 步骤 1 完成 ---


--- 步骤 2: 初始化并运行 Indexer ---
  - 正在初始化 Indexer...
创建了新的空 Faiss IndexIDMap2 索引 (维度: 512), 使用内积 (余弦相似度)。
  - Indexer 初始化完成。
  - 正在对加载的文档进行索引...
  - 文档索引过程完成。

Indexer 总结：索引中的总向量数: 219，数据库中的总文档数: 219
--- 步骤 2 完成 ---


--- 步骤 3: 初始化并运行 Retriever ---
  - 正在初始化 Retriever...
  - Retriever 初始化成功。
--- 3 步骤完成 ---


--- 步骤 4: 初始化并运行 Generator ---
  - 正在初始化 Generator...
  - Generator 初始化成功，使用模型: glm-4-flash-250414
--- 步骤 4 完成 ---


--- 步骤 5: 执行查询并生成响应 ---

##################################################

>>> 开始执行纯文本查询示例 (5 次) <<<
##################################################


--- 文本查询 1/5 ---
用户查询: 带隙基准电路的核心原理是什么？
  - 正在执行检索...
  - 正在编码查询...
  - 已编码文本查询。
  - 查询编码完成。
  - 正在 Faiss 索引中搜索 Top 5 个相似向量...
  - Faiss 搜索找到 5 个潜在最近邻。
  - 正在根据检索到的向量ID获取文档元数据...
  - 文档元数据获取完成。
  - 检索到 5 个文档。详细信息如下:
    1. ID: Comparator58, 得分: 0.7555
       文本: 

       图像文件: Comparator58.png
    --------